## 第1回 FR Frontier ：ファッション画像における洋服の「色」分類

* [コンテスト詳細 ビッグデータ活用ならオプトDSL DeepAnalytics](https://deepanalytics.jp/compe/36)

## preprocess
Opencvをインストール

    conda install opencv

In [6]:
import numpy as np
import pandas as pd
import cv2

In [2]:
import os, sys
current_dir = os.getcwd()
HOME_DIR = current_dir
DATA_GIVEN_DIR = HOME_DIR+"/data/given/"
#DATA_RESIZED_DIR = HOME_DIR+"/data/resized/"
DATA_RESIZED_DIR = "/input/"
DATA_MYDATA_DIR = HOME_DIR+"/data/processed/"
DATA_TRANS_DIR = HOME_DIR+"/data/transparent/"

In [3]:
width = 667
height = 667

### Transparent

In [44]:
os.makedirs(DATA_TRANS_DIR)
os.makedirs(DATA_TRANS_DIR+"train/")
os.makedirs(DATA_TRANS_DIR+"test/")

In [45]:
from PIL import Image
def trans_img(src, dst, width, height):
    img = Image.open(src, 'r')

    # 同じサイズの画像を作成
    trans = Image.new('RGBA', img.size, (0, 0, 0, 0))

    for x in range(width):
        for y in range(height):
            pixel = img.getpixel( (x, y) )
        
            # 白なら処理しない
            if pixel[0] == 255 and pixel[1] == 255 and pixel[2] == 255:
                continue
        
            # 白以外なら、用意した画像にピクセルを書き込み
            trans.putpixel( (x, y), pixel )
    # リサイズ後の画像を保存
    trans.save(dst, 'JPEG', optimize=True)

In [46]:
# リサイズしないと時間かかりすぎ！！
for i in range(0, 12399):
    src = DATA_GIVEN_DIR + 'train/' + 'train_%i.jpg'%i
    dst = DATA_TRANS_DIR + "train/" + 'train_%i.jpg'%i
    trans_img(src, dst, width, height)

KeyboardInterrupt: 

In [ ]:
for i in range(0, 9801):
    src = DATA_GIVEN_DIR + 'test/' + 'test_%i.jpg'%i
    dst = DATA_TRANS_DIR + "test/" + 'test_%i.jpg'%i
    trans_img(src, dst, width, height)

### Load Data

In [4]:
data = []
for i in range(0, 12399):
    src = DATA_RESIZED_DIR + 'train/' + 'train_%i.jpg'%i
    img = cv2.imread(src)

    histR = cv2.calcHist([img],[0],None,[256],[0,256])
    histG = cv2.calcHist([img],[1],None,[256],[0,256])
    histB = cv2.calcHist([img],[2],None,[256],[0,256])

    hist = np.concatenate([histR, histG, histB], axis=1)
    data.append(hist)

In [5]:
dataX = np.array(data)
dataX.shape

(12399, 256, 3)

### train/valid/test  作成

In [7]:
train_master = pd.read_table('/input/train_master.tsv', decimal='\t')
dataY = np.array(train_master['category_id'])
dataY = np.array(pd.get_dummies(dataY).astype('float32'))
dataY.shape

(12399, 24)

In [8]:
trainX, validX = dataX[:10000], dataX[10000:]
trainY, validY = dataY[:10000], dataY[10000:]
trainX.shape, validX.shape, trainY.shape, validY.shape

((10000, 256, 3), (2399, 256, 3), (10000, 24), (2399, 24))

## training

In [46]:
import keras
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Dropout, Conv2D, BatchNormalization, MaxPooling2D, ZeroPadding2D, Convolution2D
from keras.optimizers import Adam, Nadam
from keras.preprocessing.image import ImageDataGenerator

from keras.layers import Input, Merge
from keras.layers.merge import Concatenate
from keras.layers import Embedding, Conv1D, MaxPooling1D, SpatialDropout1D, Lambda

from keras.utils.data_utils import get_file

In [19]:
batch_size=64

In [33]:
def ConvBlock(model, layers, filters):
    for i in range(layers):
        model.add(ZeroPadding2D((1, 1)))
        model.add(Conv2D(filters, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

def FCBlock(model):
    model.add(Dense(4096, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

In [43]:
# Mean of each channel as provided by VGG researchers
vgg_mean = np.array([123.68, 116.779, 103.939]).reshape((1,1,3))

def vgg_preprocess(x):
    x = x - vgg_mean     # subtract mean
    return x[:, ::-1]    # reverse axis bgr->rgb

def VGG_16(size):
    model = Sequential()
    model.add(Lambda(vgg_preprocess, input_shape=(3,)+size, output_shape=(3,)+size))
    ConvBlock(model, 2, 64)
    ConvBlock(model, 2, 128)
    ConvBlock(model, 3, 256)
    ConvBlock(model, 3, 512)
    ConvBlock(model, 3, 512)

    fname = 'vgg16_bn_conv.h5'
    model.load_weights(get_file(fname, 'http://files.fast.ai/models/'+fname, cache_subdir='models'))
    return model


In [47]:
model = VGG_16((100, 100))

ValueError: Dimensions must be equal, but are 100 and 3 for 'lambda_13/sub' (op: 'Sub') with input shapes: [?,3,100,100], [1,1,3].

In [40]:
model.compile(loss='categorical_crossentropy', optimizer=Nadam(), metrics=['accuracy'])

NameError: name 'model' is not defined

In [11]:
model.summary()

NameError: name 'model' is not defined

In [17]:
conv2.compile(loss='categorical_crossentropy', optimizer=Nadam(), metrics=['accuracy'])
conv2.fit(trainX, trainY, epochs=50, batch_size=batch_size, validation_data=(validX, validY))

Train on 10000 samples, validate on 2399 samples
Epoch 1/50
10000/10000 [==============================] - 4s - loss: 2.1824 - acc: 0.4365 - val_loss: 1.7536 - val_acc: 0.5782
Epoch 2/50
10000/10000 [==============================] - 4s - loss: 1.6757 - acc: 0.5205 - val_loss: 1.3521 - val_acc: 0.6244
Epoch 3/50
10000/10000 [==============================] - 4s - loss: 1.5073 - acc: 0.5508 - val_loss: 1.1071 - val_acc: 0.6524
Epoch 4/50
10000/10000 [==============================] - 4s - loss: 1.4172 - acc: 0.5680 - val_loss: 1.0374 - val_acc: 0.6815
Epoch 5/50
10000/10000 [==============================] - 4s - loss: 1.3600 - acc: 0.5813 - val_loss: 0.9728 - val_acc: 0.6957
Epoch 6/50
10000/10000 [==============================] - 4s - loss: 1.3262 - acc: 0.5900 - val_loss: 0.9436 - val_acc: 0.6961
Epoch 7/50
10000/10000 [==============================] - 4s - loss: 1.3042 - acc: 0.5979 - val_loss: 0.9185 - val_acc: 0.7074
Epoch 8/50
10000/10000 [==============================] - 4s -

In [30]:
conv2.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [31]:
conv2.fit(trainX, trainY, epochs=5, batch_size=batch_size, validation_data=(validX, validY))

Train on 10000 samples, validate on 2399 samples
Epoch 1/5
10000/10000 [==============================] - 4s - loss: 2.4873 - acc: 0.3886 - val_loss: 1.2900 - val_acc: 0.5927
Epoch 2/5
10000/10000 [==============================] - 3s - loss: 1.7909 - acc: 0.5089 - val_loss: 1.1186 - val_acc: 0.6574
Epoch 3/5
10000/10000 [==============================] - 3s - loss: 1.5791 - acc: 0.5466 - val_loss: 1.0913 - val_acc: 0.6603
Epoch 4/5
10000/10000 [==============================] - 3s - loss: 1.4832 - acc: 0.5641 - val_loss: 1.0688 - val_acc: 0.6619
Epoch 5/5
10000/10000 [==============================] - 3s - loss: 1.4122 - acc: 0.5743 - val_loss: 1.0075 - val_acc: 0.6799


In [49]:
conv2.compile(loss='categorical_crossentropy', optimizer=Adam(0.01), metrics=['accuracy'])
conv2.fit(trainX, trainY, epochs=30, batch_size=batch_size, validation_data=(validX, validY))

Train on 10000 samples, validate on 2399 samples
Epoch 1/30
10000/10000 [==============================] - 5s - loss: 1.3982 - acc: 0.5715 - val_loss: 0.9850 - val_acc: 0.6857
Epoch 2/30
10000/10000 [==============================] - 3s - loss: 1.3502 - acc: 0.5826 - val_loss: 0.9256 - val_acc: 0.7045
Epoch 3/30
10000/10000 [==============================] - 3s - loss: 1.3199 - acc: 0.5842 - val_loss: 0.9540 - val_acc: 0.6828
Epoch 4/30
10000/10000 [==============================] - 3s - loss: 1.3131 - acc: 0.5950 - val_loss: 0.9464 - val_acc: 0.7103
Epoch 5/30
10000/10000 [==============================] - 3s - loss: 1.2978 - acc: 0.5970 - val_loss: 0.9284 - val_acc: 0.7099
Epoch 6/30
10000/10000 [==============================] - 3s - loss: 1.2978 - acc: 0.5921 - val_loss: 0.9365 - val_acc: 0.7020
Epoch 7/30
10000/10000 [==============================] - 3s - loss: 1.2807 - acc: 0.6022 - val_loss: 0.9191 - val_acc: 0.7115
Epoch 8/30
10000/10000 [==============================] - 3s -

In [44]:
conv2.compile(loss='categorical_crossentropy', optimizer=Adam(0.0001), metrics=['accuracy'])
conv2.fit(trainX, trainY, epochs=20, batch_size=batch_size, validation_data=(validX, validY))

Train on 10000 samples, validate on 2399 samples
Epoch 1/20
10000/10000 [==============================] - 5s - loss: 1.1637 - acc: 0.6316 - val_loss: 0.9057 - val_acc: 0.7128
Epoch 2/20
10000/10000 [==============================] - 3s - loss: 1.1623 - acc: 0.6354 - val_loss: 0.9080 - val_acc: 0.7061
Epoch 3/20
10000/10000 [==============================] - 3s - loss: 1.1654 - acc: 0.6409 - val_loss: 0.8957 - val_acc: 0.7170
Epoch 4/20
10000/10000 [==============================] - 3s - loss: 1.1672 - acc: 0.6397 - val_loss: 0.8982 - val_acc: 0.7107
Epoch 5/20
10000/10000 [==============================] - 3s - loss: 1.1458 - acc: 0.6402 - val_loss: 0.9142 - val_acc: 0.7145
Epoch 6/20
10000/10000 [==============================] - 3s - loss: 1.1473 - acc: 0.6379 - val_loss: 0.8978 - val_acc: 0.7145
Epoch 7/20
10000/10000 [==============================] - 3s - loss: 1.1697 - acc: 0.6338 - val_loss: 0.8971 - val_acc: 0.7132
Epoch 8/20
10000/10000 [==============================] - 3s -

In [35]:
# model.save_weights('2.h5')

In [ ]:
# model.load_weights('1.h5')

## Predict

In [18]:
data = []
for i in range(0, 9801):
    src = DATA_RESIZED_DIR + 'test/' + 'test_%i.jpg'%i
    img = cv2.imread(src)

    histR = cv2.calcHist([img],[0],None,[256],[0,256])
    histG = cv2.calcHist([img],[1],None,[256],[0,256])
    histB = cv2.calcHist([img],[2],None,[256],[0,256])

    hist = np.concatenate([histR, histG, histB], axis=1)
    data.append(hist)

In [19]:
testX = np.array(data)
testX.shape

(9801, 256, 3)

In [21]:
test_features = conv2.predict(testX, batch_size=batch_size)

In [22]:
test_labels = np.argmax(test_features, axis=1)
test_labels[:5]
#test_features[:5]
#array([20, 14, 17, 20, 23])

array([4, 3, 3, 5, 5])

In [23]:
filenames = []
for i in range(0, 9801):
    filenames.append("test_%i.jpg"%i)

In [24]:
submission = pd.DataFrame({"Image": filenames,
                             "Label": test_labels})

In [25]:
submission_file_name = 'submission_cnn3.csv'
#np.savetxt(submission_file_name, subm, delimiter=',')
submission.to_csv(submission_file_name, index=False, header=None)

In [26]:
from IPython.display import FileLink
FileLink(submission_file_name)

/output/submission_cnn3.csv